# 03_手動版_寫作智能體

## 🎯 學習目標
- 實作 STORM 第二、三階段：大綱生成與內容撰寫
- 開發結構化大綱生成功能
- 建立段落內容創作機制
- 實現風格一致性控制
- 整合引用與來源標註

---

## 1. 環境準備

### 載入必要套件與前置設定

In [ ]:
# 基礎套件
import os
import json
import re
from datetime import datetime
from typing import List, Dict, Any, Optional

# LangChain 相關
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# 環境變數
from dotenv import load_dotenv
load_dotenv()

# 初始化 LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=os.getenv('OPENAI_API_KEY')
)

print("✅ 環境準備完成")

### 載入研究結果 (模擬數據)

In [ ]:
# 模擬研究結果數據結構
sample_research_data = {
    "topic": "ChatGPT對傳統教育模式的影響",
    "research_date": datetime.now().isoformat(),
    "expert_questions": {
        "技術專家": ["ChatGPT的技術架構如何支援教育應用？"],
        "商業分析師": ["ChatGPT在教育市場的商業模式有哪些？"],
        "用戶體驗專家": ["學生使用ChatGPT學習的體驗如何？"],
        "學術研究者": ["ChatGPT對學習成效的影響有哪些研究證據？"],
        "政策制定者": ["使用ChatGPT在教育中需要哪些政策規範？"]
    },
    "comprehensive_report": """
# ChatGPT對傳統教育模式的影響 - 綜合研究報告

## 關鍵發現摘要
• ChatGPT為個性化學習提供了新的可能性，能夠根據學生需求調整教學內容
• 技術上ChatGPT具備24/7可用性，能夠即時回答學生問題並提供學習支援
• 商業模式正在快速發展，包括訂閱制教育平台和客製化教學解決方案
• 學生普遍對AI輔助學習表現出積極態度，但也存在對過度依賴的擔憂

## 重要要點
• 提升學習效率：即時回答問題，減少等待時間
• 個性化教學：根據學生程度調整解釋方式
• 創意激發：幫助學生腦力激盪和創意寫作
• 語言學習：提供多語言對話練習機會
• 批判思維：需要培養學生驗證AI回答的能力
• 倫理考量：避免學術不誠實和過度依賴
• 教師角色轉變：從知識傳授者變成學習引導者
• 評估方式改變：需要重新設計考評方法

## 專家洞察

從技術角度來看，ChatGPT的transformer架構使其能夠理解上下文並生成連貫的回應，這為教育應用提供了強大的基礎。其多模態能力的發展將進一步擴展教育應用場景。

從商業角度分析，教育科技市場正在快速擴張，ChatGPT相關的教育應用預計將帶來數十億美元的市場機會。訂閱制和企業級解決方案將成為主要商業模式。

從政策角度考量，需要建立明確的AI使用指引，確保學術誠信，同時避免擴大數位落差。隱私保護和數據安全也是重要議題。
""",
    "statistics": {
        "total_questions": 5,
        "total_sources": 10,
        "average_confidence": 0.85
    }
}

print("✅ 模擬研究數據已載入")
print(f"主題：{sample_research_data['topic']}")
print(f"問題數：{sample_research_data['statistics']['total_questions']}")

---

## 2. 大綱生成器

### 實作結構化大綱生成

In [ ]:
class OutlineGenerator:
    """大綱生成器 - 將研究結果轉換為結構化大綱"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def generate_outline(self, research_data: Dict[str, Any], 
                        article_type: str = "academic", 
                        target_length: int = 3000) -> Dict[str, Any]:
        """生成文章大綱"""
        
        print(f"📋 生成大綱：{research_data['topic']}")
        print(f"   文章類型：{article_type}")
        print(f"   目標長度：{target_length} 字")
        
        # 建立大綱生成提示詞
        outline_prompt = self._create_outline_prompt(
            research_data, article_type, target_length
        )
        
        try:
            response = self.llm.invoke([HumanMessage(content=outline_prompt)])
            outline_text = response.content
            
            # 解析大綱結構
            parsed_outline = self._parse_outline(outline_text)
            
            # 計算各部分字數分配
            outline_with_allocation = self._allocate_word_counts(parsed_outline, target_length)
            
            result = {
                'topic': research_data['topic'],
                'article_type': article_type,
                'target_length': target_length,
                'outline_structure': outline_with_allocation,
                'raw_outline': outline_text,
                'generated_at': datetime.now().isoformat()
            }
            
            print(f"✅ 大綱生成完成，共 {len(outline_with_allocation)} 個主要章節")
            return result
            
        except Exception as e:
            print(f"❌ 大綱生成失敗：{str(e)}")
            return self._create_fallback_outline(research_data, target_length)
    
    def _create_outline_prompt(self, research_data: Dict, article_type: str, target_length: int) -> str:
        """建立大綱生成提示詞"""
        
        # 提取關鍵信息
        topic = research_data['topic']
        report = research_data['comprehensive_report']
        
        # 根據文章類型調整要求
        type_requirements = {
            "academic": "學術性、嚴謹、有邏輯層次、包含引言和結論",
            "blog": "易讀、有吸引力、包含實例、適合一般讀者",
            "report": "專業、結構完整、包含摘要和建議",
            "tutorial": "循序漸進、實用、包含步驟和範例"
        }
        
        requirements = type_requirements.get(article_type, type_requirements["academic"])
        
        prompt = f"""
你是一位專業的文章架構師。請根據以下研究報告，為主題「{topic}」設計一個{article_type}風格的文章大綱。

研究報告內容：
{report[:2000]}  # 限制長度避免超出token限制

文章要求：
- 文章類型：{article_type}
- 風格特色：{requirements}
- 目標長度：約{target_length}字
- 目標讀者：對此主題感興趣的專業人士和學習者

請設計一個結構清晰、邏輯完整的大綱。大綱格式如下：

# 文章標題

## 1. 章節名稱
### 1.1 小節名稱
- 要點說明（2-3句話描述此部分要涵蓋的內容）
### 1.2 小節名稱
- 要點說明

## 2. 章節名稱
### 2.1 小節名稱
- 要點說明

（繼續其他章節...）

請確保：
1. 大綱涵蓋研究報告的主要發現
2. 章節之間有清晰的邏輯關係
3. 每個小節都有具體的內容描述
4. 適合{target_length}字左右的文章長度
"""
        return prompt
    
    def _parse_outline(self, outline_text: str) -> List[Dict[str, Any]]:
        """解析大綱文本結構"""
        sections = []
        current_section = None
        current_subsection = None
        
        lines = outline_text.split('\n')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # 主標題
            if line.startswith('# '):
                continue  # 跳過文章標題
            
            # 章節標題 (## 開頭)
            elif line.startswith('## '):
                if current_section:
                    sections.append(current_section)
                
                current_section = {
                    'title': line[3:].strip(),
                    'subsections': [],
                    'estimated_words': 0
                }
                current_subsection = None
            
            # 小節標題 (### 開頭)
            elif line.startswith('### '):
                if current_section:
                    current_subsection = {
                        'title': line[4:].strip(),
                        'description': '',
                        'estimated_words': 0
                    }
                    current_section['subsections'].append(current_subsection)
            
            # 要點描述 (- 開頭)
            elif line.startswith('- ') and current_subsection:
                current_subsection['description'] += line[2:].strip() + ' '
        
        # 加入最後一個章節
        if current_section:
            sections.append(current_section)
        
        return sections
    
    def _allocate_word_counts(self, outline_structure: List[Dict], target_length: int) -> List[Dict]:
        """為大綱分配字數"""
        total_sections = len(outline_structure)
        if total_sections == 0:
            return outline_structure
        
        # 基本字數分配
        base_words_per_section = target_length // total_sections
        
        for section in outline_structure:
            subsection_count = len(section['subsections'])
            if subsection_count > 0:
                words_per_subsection = base_words_per_section // subsection_count
                
                for subsection in section['subsections']:
                    subsection['estimated_words'] = words_per_subsection
                
                section['estimated_words'] = base_words_per_section
            else:
                section['estimated_words'] = base_words_per_section
        
        return outline_structure
    
    def _create_fallback_outline(self, research_data: Dict, target_length: int) -> Dict[str, Any]:
        """建立備用大綱"""
        topic = research_data['topic']
        
        fallback_outline = [
            {
                'title': '1. 引言與背景',
                'subsections': [
                    {'title': '1.1 研究背景', 'description': '介紹研究主題的重要性', 'estimated_words': target_length // 6},
                    {'title': '1.2 研究目的', 'description': '說明研究的目標和意義', 'estimated_words': target_length // 6}
                ],
                'estimated_words': target_length // 3
            },
            {
                'title': '2. 主要發現與分析',
                'subsections': [
                    {'title': '2.1 關鍵發現', 'description': '總結主要研究結果', 'estimated_words': target_length // 6},
                    {'title': '2.2 深度分析', 'description': '詳細分析研究發現的意義', 'estimated_words': target_length // 6}
                ],
                'estimated_words': target_length // 3
            },
            {
                'title': '3. 結論與建議',
                'subsections': [
                    {'title': '3.1 總結', 'description': '總結主要觀點', 'estimated_words': target_length // 6},
                    {'title': '3.2 未來展望', 'description': '提出未來發展建議', 'estimated_words': target_length // 6}
                ],
                'estimated_words': target_length // 3
            }
        ]
        
        return {
            'topic': topic,
            'article_type': 'academic',
            'target_length': target_length,
            'outline_structure': fallback_outline,
            'raw_outline': '備用大綱',
            'generated_at': datetime.now().isoformat()
        }

# 建立大綱生成器
outline_generator = OutlineGenerator(llm)
print("✅ 大綱生成器已建立")

### 測試大綱生成

In [ ]:
# 生成文章大綱
article_outline = outline_generator.generate_outline(
    research_data=sample_research_data,
    article_type="academic",
    target_length=2500
)

# 顯示大綱結構
print("\n📋 生成的文章大綱：")
print("=" * 50)

for i, section in enumerate(article_outline['outline_structure'], 1):
    print(f"\n## {section['title']} ({section['estimated_words']}字)")
    
    for j, subsection in enumerate(section['subsections'], 1):
        print(f"   ### {subsection['title']} ({subsection['estimated_words']}字)")
        print(f"       - {subsection['description'][:100]}...")

---

## 3. 內容生成器

### 實作段落內容創作

In [ ]:
class ContentGenerator:
    """內容生成器 - 根據大綱生成具體文章內容"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def generate_article_content(self, outline_data: Dict[str, Any], 
                                research_data: Dict[str, Any],
                                writing_style: Dict[str, str] = None) -> Dict[str, Any]:
        """生成完整文章內容"""
        
        print(f"✍️ 開始生成文章內容：{outline_data['topic']}")
        
        # 預設寫作風格
        if writing_style is None:
            writing_style = {
                'tone': 'professional',  # professional, casual, academic
                'perspective': 'third_person',  # first_person, third_person
                'citation_style': 'inline',  # inline, footnote, endnote
                'target_audience': 'professionals'  # general, students, professionals
            }
        
        # 生成文章標題
        article_title = self._generate_title(outline_data['topic'], writing_style)
        
        # 生成各章節內容
        article_sections = []
        
        for section_info in outline_data['outline_structure']:
            print(f"   📝 生成章節：{section_info['title']}")
            
            section_content = self._generate_section_content(
                section_info, research_data, writing_style, outline_data['topic']
            )
            
            article_sections.append(section_content)
        
        # 組裝完整文章
        full_article = self._assemble_article(article_title, article_sections)
        
        result = {
            'title': article_title,
            'content': full_article,
            'sections': article_sections,
            'word_count': len(full_article.split()),
            'writing_style': writing_style,
            'generated_at': datetime.now().isoformat(),
            'metadata': {
                'topic': outline_data['topic'],
                'target_length': outline_data['target_length'],
                'actual_length': len(full_article.split())
            }
        }
        
        print(f"\n✅ 文章生成完成！")
        print(f"   標題：{article_title}")
        print(f"   字數：{result['word_count']} 字")
        print(f"   章節：{len(article_sections)} 個")
        
        return result
    
    def _generate_title(self, topic: str, style: Dict[str, str]) -> str:
        """生成文章標題"""
        
        title_prompt = f"""
請為主題「{topic}」生成一個吸引人且專業的文章標題。

要求：
- 語調：{style['tone']}
- 目標讀者：{style['target_audience']}
- 標題要準確反映主題內容
- 標題要有吸引力但不誇大
- 長度適中（10-20個字）

請只回答標題，不要其他說明：
"""
        
        try:
            response = self.llm.invoke([HumanMessage(content=title_prompt)])
            return response.content.strip()
        except:
            return f"{topic}：深度分析與探討"  # 備用標題
    
    def _generate_section_content(self, section_info: Dict[str, Any], 
                                 research_data: Dict[str, Any],
                                 style: Dict[str, str],
                                 main_topic: str) -> Dict[str, Any]:
        """生成單一章節內容"""
        
        section_title = section_info['title']
        target_words = section_info['estimated_words']
        
        # 為每個小節生成內容
        subsection_contents = []
        
        for subsection in section_info['subsections']:
            subsection_content = self._generate_subsection_content(
                subsection, research_data, style, main_topic
            )
            subsection_contents.append(subsection_content)
        
        # 組合章節內容
        section_text = f"\n\n## {section_title}\n\n"
        
        for sub_content in subsection_contents:
            section_text += sub_content + "\n\n"
        
        return {
            'title': section_title,
            'content': section_text,
            'subsections': subsection_contents,
            'word_count': len(section_text.split()),
            'target_words': target_words
        }
    
    def _generate_subsection_content(self, subsection_info: Dict[str, Any],
                                    research_data: Dict[str, Any],
                                    style: Dict[str, str],
                                    main_topic: str) -> str:
        """生成小節內容"""
        
        subsection_title = subsection_info['title']
        description = subsection_info['description']
        target_words = subsection_info['estimated_words']
        
        # 提取相關研究內容
        research_context = research_data['comprehensive_report']
        
        content_prompt = f"""
你是一位專業的內容創作者。請根據以下資訊撰寫文章小節內容。

主題：{main_topic}
小節標題：{subsection_title}
小節要求：{description}
目標字數：約{target_words}字

參考研究資料：
{research_context[:1500]}

寫作風格：
- 語調：{style['tone']}
- 視角：{style['perspective']}
- 目標讀者：{style['target_audience']}

請撰寫內容，要求：
1. 內容要有邏輯性和說服力
2. 適當引用研究資料中的重點
3. 保持與整體主題的相關性
4. 語言要清晰易懂
5. 包含具體例子或數據支持論點

格式：
### {subsection_title}

[內容正文]

請開始撰寫：
"""
        
        try:
            response = self.llm.invoke([HumanMessage(content=content_prompt)])
            return response.content.strip()
        except Exception as e:
            print(f"   ⚠️ 小節生成失敗：{str(e)}")
            return f"### {subsection_title}\n\n[內容生成中遇到問題，請重新嘗試]\n"
    
    def _assemble_article(self, title: str, sections: List[Dict[str, Any]]) -> str:
        """組裝完整文章"""
        
        article = f"# {title}\n\n"
        
        # 加入生成資訊
        article += f"*本文章由AI輔助生成，生成時間：{datetime.now().strftime('%Y-%m-%d %H:%M')}*\n\n"
        
        # 加入各章節
        for section in sections:
            article += section['content']
        
        return article

# 建立內容生成器
content_generator = ContentGenerator(llm)
print("✅ 內容生成器已建立")

### 測試內容生成

In [ ]:
# 設定寫作風格
writing_style = {
    'tone': 'professional',
    'perspective': 'third_person',
    'citation_style': 'inline',
    'target_audience': 'professionals'
}

# 生成文章內容
print("🚀 開始生成文章內容...")
article_content = content_generator.generate_article_content(
    outline_data=article_outline,
    research_data=sample_research_data,
    writing_style=writing_style
)

### 預覽生成的文章

In [ ]:
# 顯示文章基本資訊
print("📄 文章資訊：")
print(f"標題：{article_content['title']}")
print(f"字數：{article_content['word_count']} 字")
print(f"章節數：{len(article_content['sections'])} 個")
print(f"目標字數：{article_content['metadata']['target_length']} 字")

# 顯示文章開頭
print("\n📖 文章預覽（前500字）：")
print("=" * 60)
preview_content = article_content['content'][:500]
print(preview_content + "...")

# 顯示章節結構
print("\n📋 章節結構：")
for i, section in enumerate(article_content['sections'], 1):
    print(f"{i}. {section['title']} - {section['word_count']}字")

---

## 4. 品質檢查器

### 實作內容品質檢查與優化

In [ ]:
class QualityChecker:
    """品質檢查器 - 檢查和優化文章品質"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def check_article_quality(self, article_data: Dict[str, Any]) -> Dict[str, Any]:
        """檢查文章品質"""
        
        print("🔍 開始品質檢查...")
        
        article_content = article_data['content']
        
        # 執行各項檢查
        checks = {
            'readability': self._check_readability(article_content),
            'coherence': self._check_coherence(article_content),
            'completeness': self._check_completeness(article_data),
            'style_consistency': self._check_style_consistency(article_content),
            'structure': self._check_structure(article_data)
        }
        
        # 計算總體品質分數
        overall_score = sum(checks.values()) / len(checks)
        
        # 生成改進建議
        suggestions = self._generate_improvement_suggestions(checks, article_content)
        
        result = {
            'overall_score': overall_score,
            'detailed_scores': checks,
            'suggestions': suggestions,
            'check_date': datetime.now().isoformat(),
            'article_stats': {
                'word_count': article_data['word_count'],
                'section_count': len(article_data['sections']),
                'avg_section_length': article_data['word_count'] / len(article_data['sections'])
            }
        }
        
        print(f"✅ 品質檢查完成！總體分數：{overall_score:.2f}/1.0")
        
        return result
    
    def _check_readability(self, content: str) -> float:
        """檢查可讀性"""
        # 簡化的可讀性檢查
        sentences = content.split('。')
        words = content.split()
        
        if len(sentences) == 0:
            return 0.0
        
        avg_sentence_length = len(words) / len(sentences)
        
        # 理想句子長度約15-25字
        if 15 <= avg_sentence_length <= 25:
            readability_score = 1.0
        elif 10 <= avg_sentence_length <= 35:
            readability_score = 0.8
        else:
            readability_score = 0.6
        
        return readability_score
    
    def _check_coherence(self, content: str) -> float:
        """檢查連貫性"""
        # 使用 LLM 評估連貫性
        coherence_prompt = f"""
請評估以下文章的邏輯連貫性和結構完整性。

文章內容（前1000字）：
{content[:1000]}

請從以下角度評估並給出0-10分的評分：
1. 段落之間的邏輯連接是否自然
2. 論點發展是否有條理
3. 整體結構是否清晰

請只回答一個0-10的數字分數：
"""
        
        try:
            response = self.llm.invoke([HumanMessage(content=coherence_prompt)])
            score_text = response.content.strip()
            
            # 提取分數
            import re
            score_match = re.search(r'\d+(?:\.\d+)?', score_text)
            if score_match:
                score = float(score_match.group())
                return min(score / 10.0, 1.0)
            else:
                return 0.7  # 預設分數
        except:
            return 0.7
    
    def _check_completeness(self, article_data: Dict[str, Any]) -> float:
        """檢查完整性"""
        target_length = article_data['metadata']['target_length']
        actual_length = article_data['word_count']
        
        # 字數完整度
        length_ratio = actual_length / target_length
        
        if 0.8 <= length_ratio <= 1.2:
            length_score = 1.0
        elif 0.6 <= length_ratio <= 1.5:
            length_score = 0.8
        else:
            length_score = 0.5
        
        # 結構完整度
        section_count = len(article_data['sections'])
        if section_count >= 3:
            structure_score = 1.0
        elif section_count >= 2:
            structure_score = 0.8
        else:
            structure_score = 0.5
        
        return (length_score + structure_score) / 2
    
    def _check_style_consistency(self, content: str) -> float:
        """檢查風格一致性"""
        # 簡化的風格一致性檢查
        
        # 檢查人稱一致性
        first_person_count = content.count('我') + content.count('我們')
        third_person_indicators = content.count('他') + content.count('她') + content.count('它們')
        
        total_words = len(content.split())
        
        if total_words > 0:
            person_inconsistency = abs(first_person_count - third_person_indicators) / total_words
            style_score = max(0.5, 1.0 - person_inconsistency * 100)
        else:
            style_score = 0.5
        
        return min(style_score, 1.0)
    
    def _check_structure(self, article_data: Dict[str, Any]) -> float:
        """檢查結構品質"""
        sections = article_data['sections']
        
        if not sections:
            return 0.0
        
        # 檢查字數分佈是否平衡
        word_counts = [section['word_count'] for section in sections]
        avg_words = sum(word_counts) / len(word_counts)
        
        # 計算標準差
        variance = sum((count - avg_words) ** 2 for count in word_counts) / len(word_counts)
        std_dev = variance ** 0.5
        
        # 標準差越小，分佈越均勻
        if avg_words > 0:
            balance_ratio = std_dev / avg_words
            balance_score = max(0.5, 1.0 - balance_ratio)
        else:
            balance_score = 0.5
        
        return min(balance_score, 1.0)
    
    def _generate_improvement_suggestions(self, checks: Dict[str, float], content: str) -> List[str]:
        """生成改進建議"""
        suggestions = []
        
        if checks['readability'] < 0.7:
            suggestions.append("建議調整句子長度，讓文章更易讀")
        
        if checks['coherence'] < 0.7:
            suggestions.append("建議加強段落間的邏輯連接")
        
        if checks['completeness'] < 0.8:
            suggestions.append("建議調整文章長度或增加內容完整性")
        
        if checks['style_consistency'] < 0.7:
            suggestions.append("建議檢查並統一寫作風格")
        
        if checks['structure'] < 0.7:
            suggestions.append("建議平衡各章節的內容長度")
        
        if not suggestions:
            suggestions.append("文章品質良好，無需特別改進")
        
        return suggestions

# 建立品質檢查器
quality_checker = QualityChecker(llm)
print("✅ 品質檢查器已建立")

### 測試品質檢查

In [ ]:
# 執行品質檢查
quality_report = quality_checker.check_article_quality(article_content)

# 顯示品質報告
print("\n📊 品質檢查報告：")
print("=" * 40)
print(f"總體品質分數：{quality_report['overall_score']:.2f}/1.0")

print("\n詳細評分：")
for metric, score in quality_report['detailed_scores'].items():
    print(f"  {metric}: {score:.2f}")

print("\n改進建議：")
for i, suggestion in enumerate(quality_report['suggestions'], 1):
    print(f"  {i}. {suggestion}")

print("\n文章統計：")
stats = quality_report['article_stats']
print(f"  總字數：{stats['word_count']}")
print(f"  章節數：{stats['section_count']}")
print(f"  平均章節長度：{stats['avg_section_length']:.0f} 字")

---

## 5. 完整寫作智能體

### 整合所有寫作功能

In [ ]:
class WritingAgent:
    """寫作智能體 - 整合大綱生成、內容創作、品質檢查"""
    
    def __init__(self, llm):
        self.llm = llm
        self.outline_generator = OutlineGenerator(llm)
        self.content_generator = ContentGenerator(llm)
        self.quality_checker = QualityChecker(llm)
    
    def write_article(self, research_data: Dict[str, Any],
                     article_config: Dict[str, Any] = None) -> Dict[str, Any]:
        """完整的文章寫作流程"""
        
        print(f"📝 開始寫作流程：{research_data['topic']}")
        print("=" * 60)
        
        # 預設配置
        if article_config is None:
            article_config = {
                'article_type': 'academic',
                'target_length': 2500,
                'writing_style': {
                    'tone': 'professional',
                    'perspective': 'third_person',
                    'citation_style': 'inline',
                    'target_audience': 'professionals'
                }
            }
        
        # 第一階段：生成大綱
        print("\n📋 階段 1: 生成文章大綱")
        outline_data = self.outline_generator.generate_outline(
            research_data=research_data,
            article_type=article_config['article_type'],
            target_length=article_config['target_length']
        )
        
        # 第二階段：生成內容
        print("\n✍️ 階段 2: 生成文章內容")
        article_content = self.content_generator.generate_article_content(
            outline_data=outline_data,
            research_data=research_data,
            writing_style=article_config['writing_style']
        )
        
        # 第三階段：品質檢查
        print("\n🔍 階段 3: 品質檢查與優化")
        quality_report = self.quality_checker.check_article_quality(article_content)
        
        # 第四階段：最終整理
        print("\n📦 階段 4: 最終整理")
        final_result = self._finalize_article(
            research_data, outline_data, article_content, quality_report, article_config
        )
        
        print(f"\n🎉 文章寫作完成！")
        print(f"   標題：{final_result['title']}")
        print(f"   字數：{final_result['word_count']} 字")
        print(f"   品質分數：{final_result['quality_score']:.2f}/1.0")
        
        return final_result
    
    def _finalize_article(self, research_data: Dict, outline_data: Dict, 
                         article_content: Dict, quality_report: Dict,
                         article_config: Dict) -> Dict[str, Any]:
        """最終整理文章"""
        
        # 加入元數據
        final_article = {
            'title': article_content['title'],
            'content': article_content['content'],
            'word_count': article_content['word_count'],
            'quality_score': quality_report['overall_score'],
            'created_at': datetime.now().isoformat(),
            
            # 詳細資料
            'metadata': {
                'source_topic': research_data['topic'],
                'article_type': article_config['article_type'],
                'target_length': article_config['target_length'],
                'actual_length': article_content['word_count'],
                'completion_ratio': article_content['word_count'] / article_config['target_length'],
                'section_count': len(article_content['sections']),
                'writing_style': article_config['writing_style']
            },
            
            # 品質資料
            'quality_report': quality_report,
            
            # 結構資料
            'outline': outline_data,
            'sections': article_content['sections'],
            
            # 來源資料
            'research_source': {
                'research_questions': len(research_data.get('expert_questions', {})),
                'source_confidence': research_data.get('statistics', {}).get('average_confidence', 0.8)
            }
        }
        
        return final_article

# 建立完整的寫作智能體
writing_agent = WritingAgent(llm)
print("✅ 寫作智能體已建立完成")

---

## 6. 完整測試

### 執行完整寫作流程

In [ ]:
# 設定文章配置
article_config = {
    'article_type': 'academic',  # academic, blog, report, tutorial
    'target_length': 2000,       # 目標字數
    'writing_style': {
        'tone': 'professional',
        'perspective': 'third_person',
        'citation_style': 'inline',
        'target_audience': 'professionals'
    }
}

# 執行完整寫作流程
final_article = writing_agent.write_article(
    research_data=sample_research_data,
    article_config=article_config
)

### 查看最終結果

In [ ]:
# 顯示文章摘要資訊
print("📄 最終文章資訊：")
print("=" * 50)
print(f"標題：{final_article['title']}")
print(f"字數：{final_article['word_count']} 字")
print(f"品質分數：{final_article['quality_score']:.2f}/1.0")
print(f"完成率：{final_article['metadata']['completion_ratio']:.1%}")
print(f"章節數：{final_article['metadata']['section_count']}")

# 品質檢查結果
print("\n🔍 品質檢查詳情：")
for metric, score in final_article['quality_report']['detailed_scores'].items():
    status = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
    print(f"  {status} {metric}: {score:.2f}")

# 改進建議
print("\n💡 改進建議：")
for suggestion in final_article['quality_report']['suggestions']:
    print(f"  • {suggestion}")

### 保存完整文章

In [ ]:
# 保存為 Markdown 檔案
safe_title = re.sub(r'[^\w\s-]', '', final_article['title']).strip().replace(' ', '_')
markdown_filename = f"article_{safe_title}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"

try:
    with open(markdown_filename, 'w', encoding='utf-8') as f:
        f.write(final_article['content'])
        
        # 加入元數據
        f.write("\n\n---\n\n")
        f.write("## 文章元數據\n\n")
        f.write(f"- **生成時間**: {final_article['created_at']}\n")
        f.write(f"- **字數**: {final_article['word_count']}\n")
        f.write(f"- **品質分數**: {final_article['quality_score']:.2f}/1.0\n")
        f.write(f"- **文章類型**: {final_article['metadata']['article_type']}\n")
        f.write(f"- **寫作風格**: {final_article['metadata']['writing_style']['tone']}\n")
    
    print(f"✅ 文章已保存為：{markdown_filename}")
    print(f"   檔案大小：{os.path.getsize(markdown_filename)} bytes")
except Exception as e:
    print(f"❌ 保存失敗：{str(e)}")

# 保存完整資料為 JSON
json_filename = f"article_full_{safe_title}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

try:
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(final_article, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 完整資料已保存為：{json_filename}")
except Exception as e:
    print(f"❌ JSON保存失敗：{str(e)}")

---

## 🎯 本章小結

### 完成功能
✅ **結構化大綱生成**：基於研究結果生成邏輯清晰的文章大綱  
✅ **智能內容創作**：根據大綱逐章節生成高品質內容  
✅ **寫作風格控制**：支援多種寫作風格和目標受眾  
✅ **品質檢查機制**：多維度評估文章品質並提供改進建議  
✅ **完整工作流程**：從大綱到最終文章的端到端自動化  

### 技術特色
1. **模組化設計**：大綱生成、內容創作、品質檢查各自獨立
2. **配置驅動**：支援多種文章類型和寫作風格
3. **品質導向**：內建多項品質檢查指標
4. **結構化輸出**：完整的元數據和結構資訊

### 支援的文章類型
- **Academic**: 學術性文章，嚴謹邏輯
- **Blog**: 部落格文章，易讀有趣
- **Report**: 專業報告，結構完整
- **Tutorial**: 教學文件，循序漸進

### 品質評估維度
- **可讀性**: 句子長度、語言清晰度
- **連貫性**: 邏輯結構、段落銜接
- **完整性**: 字數達成、結構完整
- **風格一致性**: 語調、人稱統一
- **結構品質**: 章節平衡、組織合理

### 輸出格式
- **Markdown 文件**: 適合發布和分享
- **JSON 資料**: 包含完整元數據和結構資訊
- **品質報告**: 詳細的評估結果和改進建議

### 下一步預告
在下個 notebook「04_手動版_完整流程」中，我們將：
1. 整合研究智能體與寫作智能體
2. 建立完整的 STORM 流程
3. 實現智能體間的協作機制
4. 進行端到端的系統測試

---

*寫作智能體開發完成！現在我們擁有了一個能夠根據研究結果生成高品質長文的智能寫作系統，具備完整的大綱規劃、內容創作和品質檢查能力。*